<a href="https://colab.research.google.com/github/anitanadvikova/ScrabbleGAN_RU/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/arshjot/ScrabbleGAN.git

fatal: destination path 'ScrabbleGAN' already exists and is not an empty directory.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#!unzip drive/MyDrive/RIMES.zip

1.   Перетащить RIMES в data
2.   Изменить конфиг

```python
lexicon_file = f'/content/drive/MyDrive/words_russian_5000.txt' #строка 23
data_folder_path ='/content/ScrabbleGAN/data/RIMES/' #строка 6
num_chars = 108 #строка 51
```

3.   Изменить **data/prepare_data** строки 123-149

In [ ]:
# create char_map using training labels
with open(f'{data_folder_path}/ground_truth_training_icdar2011.tsv', 'rb') as f:
  ids = f.read().decode('utf8')
  partition_ids = [i.split()[0] for i in ids.splitlines() if len(i) > 1]
  words_raw = [(' '.join(i.split()[1:])) for i in ids.splitlines() if len(i)> 1]
# Get list of unique characters and create dictionary for mapping them to integer
chars=[]
for w_i in words_raw:
  for i in w_i:
    chars.append(i)
chars=np.unique(chars)

chars=np.unique(chars)
char_map = {value: idx + 1 for (idx, value) in enumerate(chars)}
char_map['<BLANK>'] = 0
num_chars = len(char_map.keys())
  
# Extract IDs for required set
with open(f'{data_folder_path}/ground_truth_{partition_name}_icdar2011.tsv', 'rb') as f:
  ids = f.read().decode('utf8')
  partition_ids = [i.split()[0] for i in ids.splitlines() if len(i) > 1]
  words_raw = [(' '.join(i.split()[1:])) for i in ids.splitlines() if len(i)> 1]

word_data = {}
for img_path, label in zip(partition_ids, words_raw):
    if len(label)!=0:
      img_path = f'{data_folder_path}/{partition_name}/{img_path}'
      img, valid_img = read_image(img_path, len(label), img_h, char_w)
      img_id = img_path[img_path.rfind('/')+1:-5]
      if valid_img:
          try:
              word_data[img_id] = [[char_map[char] for char in label], img]
          except KeyError:
              pass

In [13]:
%cd ScrabbleGAN/data

[Errno 2] No such file or directory: 'ScrabbleGAN/data'
/content/ScrabbleGAN


In [18]:
!python prepare_data.py

Processing Data:

Number of images = 17913
Number of unique characters = 108

Data processing completed


In [ ]:
#!zip -r "/content/drive/MyDrive/RIMES_tr_data.zip" "RIMES_tr_data.pkl"

  adding: RIMES_tr_data.pkl (deflated 31%)


In [19]:
%cd ..

/content/ScrabbleGAN


Меняем `models/ScrabbleGAN.py`

```python
#строки 78-87
fake_words = []
        
with open(cfg.lexicon_file, 'r') as f:
          fake_words.extend(f.read().splitlines())
        
self.fake_words = fake_words 
```

Добавляем в  `utils/training_utils.py`

```python
#строка 26
def saved(self, model, epoch, G_opt, D_opt, R_opt, G_sch=None, D_sch=None, R_sch=None):
        filename = os.path.join('/content/drive/MyDrive', f'model_checkpoint_epoch_{epoch}.pth.tar')
        save_dict = {
            'model': model.state_dict(),
            'G_opt': G_opt.state_dict(),
            'D_opt': D_opt.state_dict(),
            'R_opt': R_opt.state_dict(),
            'epoch': epoch,
            'G_sch': G_sch.state_dict(),
            'D_sch': D_sch.state_dict(),
            'R_sch': R_sch.state_dict()
        }
        torch.save(save_dict, filename)
```

Добавляем в  `train.py`

```python
#строка 254
self.model_checkpoint.saved(self.model, epoch, self.G_optimizer, self.D_optimizer, self.R_optimizer,
                                       *self.schedulers)
```

In [26]:
!python train.py

 Loading Data 
 Model: ScrabbleGAN 
NumExpr defaulting to 2 threads.
Param count for Gs initialized parameters: 30869377
Param count for Ds initialized parameters: 36398401
ScrabbleGAN(
  (R): Recognizer(
    (convs): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (3): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3),

In [31]:
#!zip "weights/model_checkpoint_epoch_14.pth.tar.zip" "/content/drive/My Drive/"


zip error: Nothing to do! (weights/model_checkpoint_epoch_14.pth.tar.zip)


In [39]:
#%rm -rf fid_calc
!bash  python calculate_metrics.py -c '/content/ScrabbleGAN/weights/model_checkpoint_epoch_14.pth.tar'

Generating and saving fake images
Model: ScrabbleGAN
Param count for Gs initialized parameters: 30869377
Param count for Ds initialized parameters: 36398401
100% 50/50 [00:02<00:00, 20.24it/s]
Sampling and saving real images
100% 50/50 [00:00<00:00, 114.51it/s]
/usr/bin/python3: No module named pytorch_fid


In [44]:
!python -m pytorch_fid '/content/ScrabbleGAN/fid_calc/real' '/content/ScrabbleGAN/fid_calc/fake'

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:00<00:00, 138MB/s]
100% 1/1 [00:03<00:00,  3.61s/it]
100% 1/1 [00:01<00:00,  1.01s/it]
FID:  225.24632311114516
